In [7]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import numpy as np

files = glob.glob('C:/Windows/System32/111DATASCIENCE/rollingsales_*.xls')
file_name="rollingsales_*.xls"
df = pd.concat((pd.read_excel(file,engine='openpyxl') for file in files), ignore_index=True)

# Додаємо ті стовпці які будемо аналізувати
all_data = df[['BOROUGH', 'NEIGHBORHOOD','BUILDING CLASS CATEGORY','TAX CLASS AT PRESENT','BUILDING CLASS AT PRESENT','TOTAL UNITS','LAND SQUARE FEET','GROSS SQUARE FEET','YEAR BUILT','TAX CLASS AT TIME OF SALE','BUILDING CLASS AT TIME OF SALE','SALE PRICE','SALE DATE']].copy()


# Замінюємо значення  BOROUGH з цифр на назви 
all_data.loc[:,'BOROUGH'] = all_data['BOROUGH'].replace({1: 'Manhattan', 2: 'Bronx', 3: 'Brooklyn', 4: 'Queens', 5: 'Staten Island'})

 
all_data['SALE DATE'] = pd.to_datetime(all_data['SALE DATE'], format='%Y-%m-%d')
all_data['TAX CLASS AT PRESENT'] = all_data['TAX CLASS AT PRESENT'].str.strip()


all_data = all_data.dropna(subset=['SALE PRICE'])  # Вилучення рядків з NaN
all_data = all_data[all_data['SALE PRICE'] != 0]

# Замінюємо 0 та NaN значення
all_data = all_data.dropna(subset=['SALE PRICE'])  
all_data = all_data[all_data['SALE PRICE'] != 0]
min_land_square_feet = all_data[all_data['LAND SQUARE FEET']> 0]['LAND SQUARE FEET'].min()
min_gross_square_feet = all_data[all_data['GROSS SQUARE FEET'] > 0]['GROSS SQUARE FEET'].min()
# min_gross_sale_price = all_data[all_data['SALE PRICE'] > 0]['SALE PRICE'].min()

all_data['LAND SQUARE FEET'] = all_data['LAND SQUARE FEET'].fillna(min_land_square_feet).astype(int)
all_data['GROSS SQUARE FEET'] = all_data['GROSS SQUARE FEET'].fillna(min_gross_square_feet).astype(int)
# all_data['SALE PRICE'] = all_data['SALE PRICE'].fillna(min_gross_sale_price).astype(int)
# all_data.loc[all_data['SALE PRICE'] == 0, 'SALE PRICE'] = min_gross_sale_price
all_data['TOTAL UNITS'] = all_data['TOTAL UNITS'].fillna(1).astype(int)
all_data.loc[all_data['TOTAL UNITS'] == 0, 'TOTAL UNITS'] = 1
# all_data.loc['TOTAL UNITS'] = all_data['TOTAL UNITS'].replace(0, 1)


all_data['LAND SQUARE FEET'] = all_data['LAND SQUARE FEET'].astype(int)
all_data['GROSS SQUARE FEET'] = all_data['GROSS SQUARE FEET'].astype(int)
all_data['TOTAL UNITS'] = all_data['TOTAL UNITS'].astype(int)
all_data['SALE PRICE'] = all_data['SALE PRICE'].astype(int)

# Додаємо стовпці BUILDING TYPE та PRICE PER SQFT
use_class_mapping = {
    'RESIDENTIAL': ['FAMILY', 'RESIDENTIAL', 'APARTMENTS'],
    'NON-RESIDENTIAL': ['OFFICE', 'STORE', 'GARDENS', 'PARKING', 'FACTORY', 'GARAGES']
}

# Функція для визначення категорії на основі значення Planning Use Class
def determine_use_class(description):
    for use_class, keywords in use_class_mapping.items():
        for keyword in keywords:
            if keyword in description.upper():
                return use_class
    return 'ELSE'

all_data['BUILDING TYPE'] = all_data['BUILDING CLASS CATEGORY'].apply(determine_use_class)
all_data['PRICE PER SQFT'] = all_data['SALE PRICE'] / all_data['GROSS SQUARE FEET']

print(all_data)

# Додаємо ті стовпці які будемо аналізувати
all_data = df[['BOROUGH', 'NEIGHBORHOOD','BUILDING CLASS CATEGORY','TAX CLASS AT PRESENT','BUILDING CLASS AT PRESENT','TOTAL UNITS','LAND SQUARE FEET','GROSS SQUARE FEET','YEAR BUILT','TAX CLASS AT TIME OF SALE','BUILDING CLASS AT TIME OF SALE','SALE PRICE','SALE DATE']].copy()


# Замінюємо значення  BOROUGH з цифр на назви 
all_data.loc[:,'BOROUGH'] = all_data['BOROUGH'].replace({1: 'Manhattan', 2: 'Bronx', 3: 'Brooklyn', 4: 'Queens', 5: 'Staten Island'})

average_price_by_borough = all_data.groupby('BOROUGH')['SALE PRICE'].mean().reset_index()
plt.bar(average_price_by_borough['BOROUGH'], average_price_by_borough['SALE PRICE'])
plt.gca().get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
plt.xlabel('Borough') 
plt.ylabel('Average selling price')
plt.title('Average selling price by Borough')
plt.show()

# Популярні та непопулярні Райони
average_prices = all_data.groupby(['BOROUGH', 'NEIGHBORHOOD'])['SALE PRICE'].mean().reset_index()

# Знаходимо район з найвищою середньою ціною в кожному BOROUGH
highest_prices = average_prices.groupby('BOROUGH')['SALE PRICE'].idxmax()
neighborhoods_with_highest_prices = average_prices.loc[highest_prices]

# Знаходимо район з найнижчою середньою ціною в кожному BOROUGH
lowest_prices = average_prices.groupby('BOROUGH')['SALE PRICE'].idxmin()
neighborhoods_with_lowest_prices = average_prices.loc[lowest_prices]

# Підготовка назв районів разом з BOROUGH для графіку
neighborhoods_highest_labels = [f'{row["NEIGHBORHOOD"]}, {row["BOROUGH"]}' for _, row in neighborhoods_with_highest_prices.iterrows()]
neighborhoods_lowest_labels = [f'{row["NEIGHBORHOOD"]}, {row["BOROUGH"]}' for _, row in neighborhoods_with_lowest_prices.iterrows()]

# Відобразимо результати на графіку
plt.figure(figsize=(12, 6))
plt.bar(neighborhoods_highest_labels, neighborhoods_with_highest_prices['SALE PRICE'], color='red', label='Highest price')
plt.bar(neighborhoods_lowest_labels, neighborhoods_with_lowest_prices['SALE PRICE'], color='blue', label='Lowest price')
plt.gca().get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
plt.xlabel('Borough')
plt.ylabel('Average selling price')
plt.title('Neighborhood with the highest and lowest average price')
plt.xticks(rotation=90)
plt.legend()
plt.show()


# Найвищі та найнижчі ціни
# Створюємо новий DataFrame, де обчислимо середню ціну для кожного BOROUGH
average_prices_by_borough = all_data.groupby('BOROUGH')['SALE PRICE'].mean().reset_index()

# Знаходимо BOROUGH з найвищою середньою ціною
highest_price_borough = average_prices_by_borough.loc[average_prices_by_borough['SALE PRICE'].idxmax()]

# Знаходимо BOROUGH з найнижчою середньою ціною
lowest_price_borough = average_prices_by_borough.loc[average_prices_by_borough['SALE PRICE'].idxmin()]

# Відобразимо результати на графіку
plt.figure(figsize=(8, 6))
plt.bar(highest_price_borough['BOROUGH'], highest_price_borough['SALE PRICE'], color='red', label='Highest prices')
plt.bar(lowest_price_borough['BOROUGH'], lowest_price_borough['SALE PRICE'], color='blue', label='Lowest prices')
plt.gca().get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
plt.xlabel('Borough')
plt.ylabel('Average selling price')
plt.title('Borough with the highest and lowest average property prices')
plt.legend()
plt.show()

# Обсяг продажів
sales_by_borough = all_data['BOROUGH'].value_counts().reset_index()
sales_by_borough.columns = ['BOROUGH', 'SALES COUNT']

# Побудова графіку
plt.figure(figsize=(10, 6))
plt.bar(sales_by_borough['BOROUGH'], sales_by_borough['SALES COUNT'])
plt.xlabel('Borough')
plt.ylabel('Sales count')
plt.title('Sales volume by each Borough')
plt.show()

# print(sales_by_borough)

# Порівняння типів будівель за кількістю продажів
building_type_counts = all_data['BUILDING CLASS CATEGORY'].value_counts()
# Групуємо дані, де кількість продажів менше 1 в "Other"
threshold = 1500
building_type_counts['Other'] = building_type_counts[building_type_counts < threshold].sum()
building_type_counts = building_type_counts[building_type_counts >= threshold]

# Побудова pie chart
plt.figure(figsize=(6, 6))
plt.pie(building_type_counts, labels=building_type_counts.index, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Робимо коло круглим
plt.title('Comparison of building types by number of sales')
plt.show()

#Кількість житлових та нежитлових будівель
building_type_counts = all_data['BUILDING TYPE'].value_counts()
building_type_countsall = all_data['BUILDING CLASS CATEGORY'].value_counts()
print(building_type_counts)
print(building_type_countsall)
# Побудова графіку
plt.figure(figsize=(8, 6))
building_type_counts.plot(kind='bar', color='skyblue')
plt.title('Number of residential and non-residential buildings')
plt.xlabel('Type')
plt.ylabel('Number')
plt.xticks(rotation=0)
plt.show()


# Порівняння обсягу продажів за різними податковими класами
sales_by_tax_class = all_data.groupby('TAX CLASS AT PRESENT')['SALE PRICE'].count().reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(x='TAX CLASS AT PRESENT', y='SALE PRICE', data=sales_by_tax_class,hue ='TAX CLASS AT PRESENT',palette='viridis')
plt.xlabel('Tax Class')
plt.ylabel('Sales count')
plt.title('Comparison of the volume of sales by different tax classes')
plt.xticks(rotation=45)
plt.show()

# Середня ціна за 1 кв. фут площі у Районі
filtered_data = all_data[(all_data['GROSS SQUARE FEET'] > 0)]
# Розрахунок ціни за 1 кв. фут площі
filtered_data.loc[:,'PRICE PER SQFT'] = filtered_data['SALE PRICE'] / filtered_data['GROSS SQUARE FEET']

# Групування та розрахунок середньої ціни за 1 кв. фут площі за типами будівель та BOROUGH
price_per_sqft_by_building = filtered_data.groupby(['BOROUGH'])['PRICE PER SQFT'].mean().reset_index()

print(price_per_sqft_by_building)
# Побудова графіку
plt.figure(figsize=(8, 6))
plt.gca().get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
sns.barplot(data=price_per_sqft_by_building, x='BOROUGH', y='PRICE PER SQFT', hue='BOROUGH')
plt.xlabel('Type')
plt.ylabel('Average price for 1 square meter. ft')
plt.title('Average price for 1 square meter. ft. of square footage by building types and Borough')
plt.xticks(rotation=90)
plt.show()

# Середня площа будівель за BOROUGH
filtered_data = all_data[all_data['GROSS SQUARE FEET'] > 0]

# Групування та розрахунок середньої площі за BOROUGH
average_sqft_by_borough = filtered_data.groupby('BOROUGH')['GROSS SQUARE FEET'].mean().reset_index()

# Побудова графіку
plt.figure(figsize=(8, 6))
sns.barplot(data=average_sqft_by_borough, x='BOROUGH', y='GROSS SQUARE FEET', hue='BOROUGH', palette='viridis')
plt.xlabel('Borough')
plt.ylabel('Average building area (sq. ft.)')
plt.title('Average area of buildings by Borough')
plt.show()

